In [6]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from random import gauss
import matplotlib.pyplot as plt
import warnings
from math import *
warnings.filterwarnings('ignore')

In [7]:
#Process options chain for one ticker from CBOE website 
with open('AAPL.dat','r') as f:
    underlying = float(f.readline().split(',')[1])
dat = pd.read_csv('/Users/Dovla/Downloads/AAPL.dat', sep=",", skiprows=2)
calls = dat.iloc[:,0:7]
calls['Y'], calls['M'],calls['D'],calls['K'],calls['Code'] = calls['Calls'].str.split(' ', -1).str
calls['Code'],calls['Exch'] = calls['Code'].str.split('-', -1).str
calls['K'] = calls.K.astype(float)
calls['Time'] = pd.to_datetime(calls.Y+calls.M+calls.D,format='%Y%b%d')
calls['Time1'] = calls['Time'] - calls['Time'][0]
calls['Ti'] = calls['Time1'].dt.days / 365
calls['S'] = underlying
calls['r'] = 0.03
calls['opMidPrice'] = (calls['Bid'] + calls['Ask'])/2
calls['sigma'] = 0.1
calls = calls[calls['Exch'].isnull()]
calls1 = calls[['S','K','Ti','r','sigma','opMidPrice','Vol']]

In [8]:
def d1(x):
    d1 = (np.log(x.S/x.K) + (x.r + x.sigma**2/2)*x.Ti) / (x.sigma * np.sqrt(x.Ti))
    return d1
def d2(x):
    d2 = x.d1 - x.sigma * np.sqrt(x.Ti)
    return d2
def call(x):
    c0 =  x.S * norm.cdf(x.d1) - x.K * exp(-x.r*(x.Ti)) * norm.cdf(x.d2)
    return c0
def deltaCall(x):
    return norm.cdf(x.d1)
def gamma(x):
    gamma = norm.pdf(x.d1) / (x.S * x.sigma * sqrt(x.Ti))
    return gamma
def thetaCall(x):
    theta = -(x.S*norm.pdf(x.d1)*x.sigma / (2*sqrt(x.Ti))) - (0.03 * x.K*exp(-0.03*x.Ti)*norm.cdf(x.d2))
    return theta
def rhoCall(x):
    rho = ((x.Ti) * x.K * exp(-0.03*(x.Ti)) * norm.cdf(x.d2))
    return rho

In [9]:
calls1['d1'] = calls1.apply(d1, axis=1)
calls1['d2'] = calls1.apply(d2, axis=1)
calls1['opBSprice'] = calls1.apply(call, axis=1)
calls1['deltaCall'] = calls1.apply(deltaCall, axis=1)
calls1['gamma'] = calls1.apply(gamma, axis=1)
calls1['thetaCall'] = calls1.apply(thetaCall, axis=1)
calls1['rhoCall'] = calls1.apply(rhoCall, axis=1)

In [10]:
def GBM(x):
    return x.S * exp((x.r - 0.5 * x.sigma**2) * x.Ti + x.sigma * np.sqrt(x.Ti) * gauss(0,1.0))
def MC(x, sims=100):
    payoffs = []
    for i in range(sims):
        S_T = GBM(x)
        payoffs.append(max(0.0,S_T-x.K))
    price = np.exp(-x.r * (x.Ti)) * (sum(payoffs) / float(sims))
    return price
calls1['CallMC'] = calls1.apply(MC, axis=1)

In [11]:
def call1(S,K,r,sigma,Ti):
    d1 = (np.log(S/K) + (r + sigma**2/2)*Ti) / (sigma * np.sqrt(Ti))
    d2 = d1 - sigma * np.sqrt(Ti)
    c0 =  S * norm.cdf(d1) - K * exp(-r*(Ti)) * norm.cdf(d2)
    return c0, d1
def vega(S,d1,Ti):
    vega = (S * norm.pdf(d1) * np.sqrt(Ti))
    return vega
def impVol(x):
    iters = 100
    epsilon = 1.0e-5
    vol = 0.5
    for i in range(0, iters):
        price, d111 = call1(x.S,x.K,x.r,vol,x.Ti)
        veg = vega(x.S,d111,x.Ti)
        diff = x.opMidPrice - price  # our root
        if (abs(diff) < epsilon):
            return vol
        vol = vol + diff/veg # f(x) / f'(x)
    return vol
calls2 = calls1.loc[calls1.Ti == calls1.Ti.iloc[501],]
calls2['ImpVol'] = calls2.apply(impVol, axis=1)